In [1]:
import os
os.chdir('/home/hyunjun/workspace/technical_indicator')

In [2]:
import talib as ta
import pandas as pd

from coinmarketcap_dataloader import CoinMarketCap
from coinmarketcap_dataloader import CMC_API_KEY

In [3]:
wrapper = CoinMarketCap(CMC_API_KEY)

In [ ]:
ticker_infos = wrapper.get_all_listings()
tickers = [info['symbol'] for info in ticker_infos]
tickers

In [17]:
from datetime import datetime, timedelta

def get_ohlcv_from_now(ticker, count=100):
    
    current_date = datetime.now()
    date_ago = current_date - timedelta(days = count + 1)

    historical_info = wrapper.get_ohlcv_historical(ticker, start=date_ago)
    df = pd.DataFrame([info['quote']['USD'] for info in historical_info])
    df = df.set_index('timestamp', drop=True)
    return df

df = get_ohlcv_from_now('BTC', count=100)

In [144]:
import numpy as np
import talib as ta

from typing import Tuple
from datetime import datetime
from datetime import timedelta


class Indicator(CoinMarketCap):
    def __init__(self, API_KEY):
        super().__init__(API_KEY)

        self.tickers:list = None
        self.historical_data_dict:dict = {}

        self.update_tickers()
        self.update_historical_data_frame()


    def update_tickers(self):
        """
        Ticker List를 멤버변수 self.tickers에 저장
        """
        ticker_infos = self.get_all_listings()
        self.tickers = [info['symbol'] for info in ticker_infos]


    def update_historical_data_frame(self):
        """
        Ticker 과거 데이터를 멤버변수 self.historical_data_frame에 저장
        """
        for ticker in self.tickers[:3]:
            self.historical_data_dict.update({ticker:self.get_ohlcv_from_now(ticker)})


    def get_total_score(self, tickers:list) -> dict:
        """
        입력 ticker들에 대한 total score list를 리턴
        """
        score_dict = {}

        scoring_func = [self.get_RSI_score,
                        self.get_MACD_score,
                        self.get_OBV_score,
                        self.get_STOCH_score,
                        self.get_BBANDS_score]

        
        for ticker in tickers:
            score_dict[ticker] = [list(func(ticker)) for func in scoring_func]

        score_array = np.array(list(score_dict.values()))

        mean_score_n1 = self._get_normalize( np.mean(score_array[:,:,0], axis=-1) )
        mean_score_n2 = self._get_normalize( np.mean(score_array[:,:,1], axis=-1) )
        mean_score_n3 = self._get_normalize( np.mean(score_array[:,:,2], axis=-1) )

        total_mean_score = self._get_normalize( np.mean([mean_score_n1, mean_score_n2, mean_score_n3], axis=-1) )

        return mean_score_n1, mean_score_n2, mean_score_n3, total_mean_score


    def get_RSI_score(self, ticker:str) -> Tuple[int, int, int]:
        """
        RSI
        """

        def score_func(rsi):
            if rsi < 30: 
                return -1
            elif rsi > 70:
                return 1
            else: 
                return 0

        n1, n2, n3 = 7, 28, 91

        data = self._get_price_to_now(ticker, ['close'])

        rsi_series_n1 = ta.RSI(data['close'], n1)
        rsi_series_n2 = ta.RSI(data['close'], n2)
        rsi_series_n3 = ta.RSI(data['close'], n3)

        rsi_n1 = rsi_series_n1[-1]
        rsi_n2 = rsi_series_n2[-1]
        rsi_n3 = rsi_series_n3[-1]

        rsi_n1_score = score_func(rsi_n1)
        rsi_n2_score = score_func(rsi_n2)
        rsi_n3_score = score_func(rsi_n3)
    
        return rsi_n1_score, rsi_n2_score, rsi_n3_score
    

    def get_MACD_score(self, ticker:str) -> Tuple[int, int, int]:
        """
        MACD 
        """
        def score_func(macd):
            if macd > 0: 
                return 1
            else:
                return -1

        n1, n2, n3 = 7, 28, 91

        data = self._get_price_to_now(ticker, ['close'])

        macd_series_n1, _, _ = ta.MACD(data['close'], fastperiod=12, slowperiod=26, signalperiod=n1)
        macd_series_n2, _, _ = ta.MACD(data['close'], fastperiod=12, slowperiod=26, signalperiod=n2)
        macd_series_n3, _, _ = ta.MACD(data['close'], fastperiod=12, slowperiod=26, signalperiod=n3)

        macd_n1 = macd_series_n1[-1]
        macd_n2 = macd_series_n2[-1]
        macd_n3 = macd_series_n3[-1]

        macd_n1_score = score_func(macd_n1)
        macd_n2_score = score_func(macd_n2)
        macd_n3_score = score_func(macd_n3)

        return macd_n1_score, macd_n2_score, macd_n3_score


    def get_BBANDS_score(self, ticker:str) -> Tuple[int, int, int]:
        """
        BBANDS 
        """
        def score_func(upper, lower, price):
            if price < lower:
                return -1
            elif price > upper:
                return 1
            else:
                return 0

        n1, n2, n3 = 7, 28, 91

        data = self._get_price_to_now(ticker, ['close'])

        upper_band_n1, _, lower_band_n1 = ta.BBANDS(data['close'], timeperiod=n1, nbdevup=1, nbdevdn=1)
        upper_band_n2, _, lower_band_n2 = ta.BBANDS(data['close'], timeperiod=n2, nbdevup=1, nbdevdn=1)
        upper_band_n3, _, lower_band_n3 = ta.BBANDS(data['close'], timeperiod=n3, nbdevup=1, nbdevdn=1)
        
        upper_n1, lower_n1 = upper_band_n1[-1], lower_band_n1[-1]
        upper_n2, lower_n2 = upper_band_n2[-1], lower_band_n2[-1]
        upper_n3, lower_n3 = upper_band_n3[-1], lower_band_n3[-1]

        bband_n1_score = score_func(upper_n1, lower_n1, data['close'][-1])
        bband_n2_score = score_func(upper_n2, lower_n2, data['close'][-1])
        bband_n3_score = score_func(upper_n3, lower_n3, data['close'][-1])

        return bband_n1_score, bband_n2_score, bband_n3_score
    
    
    def get_STOCH_score(self, ticker:str) -> Tuple[int, int, int]:
        """
        STOCH
        """

        def score_func(slowk):
            if slowk < 20:
                return -1
            elif slowk > 80:
                return 1
            else:
                return 0

        n1, n2, n3 = 7, 28, 91

        data = self._get_price_to_now(ticker, ['high', 'low', 'close'])

        slowk_series_n1, _ = ta.STOCH(data['high'], data['low'], data['close'], 
                                fastk_period=n1, slowk_period=3, slowd_period=3)
        
        slowk_series_n2, _ = ta.STOCH(data['high'], data['low'], data['close'], 
                                fastk_period=n2, slowk_period=3, slowd_period=3)
        
        slowk_series_n3, _ = ta.STOCH(data['high'], data['low'], data['close'], 
                                fastk_period=n3, slowk_period=3, slowd_period=3)
        
        slowk_n1 = slowk_series_n1[-1]
        slowk_n2 = slowk_series_n2[-1]
        slowk_n3 = slowk_series_n3[-1]

        slowk_n1_score = score_func(slowk_n1)
        slowk_n2_score = score_func(slowk_n2)
        slowk_n3_score = score_func(slowk_n3)

        return slowk_n1_score, slowk_n2_score, slowk_n3_score


    def get_OBV_score(self, ticker:str) -> Tuple[int, int, int]:
        """
        OBV
        """

        def score_func(obv):
            if obv > 0:
                return 1
            else:
                return -1

        n1, n2, n3 = 7, 28, 91

        data = self._get_price_to_now(ticker, ['close', 'volume'])

        obv_series_n1 = ta.OBV(data['close'], data['volume'])
        obv_series_n2 = ta.OBV(data['close'], data['volume'])
        obv_series_n3 = ta.OBV(data['close'], data['volume'])

        obv_n1 = obv_series_n1[-1]
        obv_n2 = obv_series_n2[-1]
        obv_n3 = obv_series_n3[-1]

        obv_n1_score = score_func(obv_n1)
        obv_n2_score = score_func(obv_n2)
        obv_n3_score = score_func(obv_n3)

        return obv_n1_score, obv_n2_score, obv_n3_score


    def get_ohlcv_from_now(self, ticker:str, count=100) -> pd.DataFrame:
        """ 
        현재 시점으로부터 과거 n개의 ohlcv 데이터를 리턴
        """
        
        date_ago = datetime.now() - timedelta(days = count + 1)
        historical_info = self.get_ohlcv_historical(ticker, start=date_ago)
        dataframe = pd.DataFrame([info['quote']['USD'] for info in historical_info])
        dataframe = dataframe.set_index('timestamp', drop=True)
        return dataframe
    

    def _get_price_to_now(self, ticker:str, types:list = ['close']) -> pd.DataFrame:
        """
        과거 데이터 + 현재 시점까지의 가격 및 거래량 데이터를 리턴
        """

        # 전일까지의 price dataframe
        historical_df = self.historical_data_dict[ticker]
        # 현재시점의 price dataframe
        realtime_df = self.get_ohlcv_realtime([ticker])
        realtime_df = pd.DataFrame([realtime_df[ticker][0]['quote']['USD']])
        realtime_df.set_index('last_updated', inplace=True)
        # price series  
        price = pd.concat([historical_df[types], realtime_df[types]])
        return price
    
    def _get_normalize(self, array:np.array, upper:int=5, lower:int=-5):
        """
        Min-Max normalize으로 upper, lower bounding
        """
        k = upper - lower
        d = -lower

        if len(array) == 1:
            return array

        new_array = array.copy()
        max_score = np.max(new_array)
        min_score = np.min(new_array)

        new_array = k * (new_array - min_score) / (max_score - min_score) - d
        return new_array

In [145]:
indicator = Indicator(CMC_API_KEY)

In [131]:
indicator.get_RSI_score('BTC')

1.0

In [132]:
indicator.get_MACD_score('BTC')

0.3333333333333333

In [133]:
indicator.get_STOCH_score('BTC')

1.0

In [134]:
indicator.get_OBV_score('BTC')

1.0

In [135]:
indicator.get_BBANDS_score('BTC')

0.6666666666666666

In [146]:
score_dict = indicator.get_total_score(indicator.tickers[:3])

In [159]:
score_array = np.array(list(score_dict.values()))
score_array.shape

(3, 5, 3)

In [167]:
mean_score_n1 = np.mean(score_array[:,:,0], axis=-1)
mean_score_n2 = np.mean(score_array[:,:,1], axis=-1)
mean_score_n3 = np.mean(score_array[:,:,2], axis=-1)

array([ 1. ,  0.8, -0.4])

In [189]:
np.mean([mean_score_n1, mean_score_n2, mean_score_n3], axis=-1)

array([0.46666667, 0.46666667, 0.2       ])